In [1]:
# hide
# default_exp utilities
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *
from nbdev.export import notebook2script

# Utilities

> Scripts and functions used by other modules

In [2]:
# export
import glob
import os
import re
import subprocess
import tempfile
from pprint import pprint

import git
from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper
from fastcore.script import *

In [3]:
# export
def _run_cmd(cmd):
    return subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True).decode("utf-8")

def _print_cmd_output(cmd):
    print(_run_cmd(cmd))

# Project metadata/configuration

Metadata and configuration for the project are stored in the `.kicad_helpers_config` directory (including [KiBot](https://github.com/INTI-CMNB/KiBot) config files for producting manufacturing outputs via continuous integration). Additional metadata (e.g., project summary, website, and manufacturing details) is stored in the project's `kitspace.yaml` file which makes it easy for other people to manufacture boards using https://kitspace.org/.

In [4]:
#export
def get_git_root(path="."):
    # Find the current projects' root directory
    git_repo = git.Repo(path, search_parent_directories=True)
    return git_repo.git.rev_parse("--show-toplevel").replace("/", os.path.sep)

In [5]:
#export
@call_parse
def setup_test_repo(root:Param("project root directory", str)="_temp"):
    """Setup a test KiCad repository to test against.
    """
    if not os.path.exists(root):
        subprocess.check_call(f"git clone --recursive https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad { root }", shell=True)

In [6]:
#hide
root = os.path.join(get_git_root("."), "_temp")
setup_test_repo(root)

In [7]:
#export
def _set_root(root):
    """If `root` is the default value ("."), use the project's git root
    or override with the environment variable `KH_PROJECT_ROOT` if it
    exists.
    """
    if root == ".":
        # Use defaults
        root = get_git_root(".")
        
        # Override with environment variable if set
        root = os.getenv("KH_PROJECT_ROOT", root)
    return root

In [8]:
#export
def get_project_name(root="."):
    """Get the project name based on the name of the KiCad `*.pro` file.
    """
    root = _set_root(root)
    return os.path.splitext(os.path.split(glob.glob(os.path.join(root, "*.pro"))[0])[1])[0]

In [9]:
#hide_input
print(f"> get_project_name()\n{ get_project_name(root) }")
assert get_project_name(root) == "40-channel-hv-switching-board"

> get_project_name()
40-channel-hv-switching-board


In [10]:
#export
def get_project_metadata(root="."):
    """Get the project metatdata from the `kitspace.yaml` file.
    """
    root = _set_root(root)

    # Default metadata if there's no existing `kitspace.yaml` file.
    metadata = {"summary": "A description for your project",
                "site": "https://example.com # A site you would like to link to (include http:// or https://)",
                "color": "black"
    }
    
    try:
        # If there's an existing `kicad.yaml` file, those settings override the defaults.
        with open(os.path.join(root, "kitspace.yaml")) as f:
            metadata.update({k: v for k, v in load(f, Loader=Loader).items() if k in ["summary", "site", "color"]})
    except FileNotFoundError:
        pass

    # Add the project name
    metadata["project_name"]=get_project_name(root)
    return metadata


In [11]:
#hide_input
print("> get_project_metadata()")
pprint(get_project_metadata(root))
assert get_project_metadata(root) == {'summary': 'DropBot v3 40-channel high-voltage switching board', 'site': 'https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad', 'color': 'black', 'project_name': '40-channel-hv-switching-board'}

> get_project_metadata()
{'color': 'black',
 'project_name': '40-channel-hv-switching-board',
 'site': 'https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad',
 'summary': 'DropBot v3 40-channel high-voltage switching board'}


In [12]:
#export
def get_schematic_path(root="."):
    """Get the path to the KiCad schematic.
    """
    root = _set_root(root)
    return os.path.join(root, get_project_name(root) + ".sch")

In [13]:
#hide_input
print(f"> get_schematic_path()\n{ get_schematic_path(root) }")
assert os.path.exists(get_schematic_path(root))

> get_schematic_path()
C:\Users\Ryan\OneDrive\dev\kicad-helpers\_temp\40-channel-hv-switching-board.sch


In [14]:
#export
def get_bom_path(root="."):
    """Get the path to the BOM.
    """
    root = _set_root(root)
    return os.path.join(root, "manufacturing", "default", get_project_name(root) + "-BOM.csv")

In [15]:
#hide_input
print(f"> get_bom_path()\n{ get_bom_path(root) }")
assert os.path.exists(get_bom_path(root))

> get_bom_path()
C:\Users\Ryan\OneDrive\dev\kicad-helpers\_temp\manufacturing\default\40-channel-hv-switching-board-BOM.csv


In [16]:
#export
def get_board_path(root="."):
    """Get the path to the KiCad board file.
    """
    root = _set_root(root)
    return os.path.join(root, get_project_name(root) + ".kicad_pcb")

In [17]:
#hide_input
print(f"> get_board_path()\n{ get_board_path(root) }")
assert os.path.exists(get_board_path(root))

> get_board_path()
C:\Users\Ryan\OneDrive\dev\kicad-helpers\_temp\40-channel-hv-switching-board.kicad_pcb


In [18]:
#export
def get_manufacturers(root="."):
    """Get the supported manufacturers.
    """
    root = _set_root(root)
    return [os.path.split(f)[1][:-5] for f in glob.glob(os.path.join(root, ".kicad_helpers_config", "manufacturers", "*.yaml"))]

In [19]:
#hide_input
print(f"> get_manufacturers()\n{ get_manufacturers(root) }")
assert get_manufacturers(root) == ['default', 'PCBWay']

> get_manufacturers()
['default', 'PCBWay']


In [20]:
#export
def get_gitignore_list(root="."):
    root =_set_root(root)
    with open(f"{ root }/.gitignore") as f:
        gitignore = f.readlines()
    return [line.strip() for line in gitignore]

In [21]:
#hide_input
print(f"> get_gitignore_list()\n{ get_gitignore_list(root) }")
assert get_gitignore_list(root) == ['_autosave*',
    '*bak',
    '*.xml',
    '.ipynb_checkpoints',
    '*-erc.txt',
    '*-drc.txt',
    'kibot_errors.filter'
]

> get_gitignore_list()
['_autosave*', '*bak', '*.xml', '.ipynb_checkpoints', '*-erc.txt', '*-drc.txt', 'kibot_errors.filter']


In [22]:
get_gitignore_list(root)

['_autosave*',
 '*bak',
 '*.xml',
 '.ipynb_checkpoints',
 '*-erc.txt',
 '*-drc.txt',
 'kibot_errors.filter']

In [23]:
#export
def in_gitignore(filename, root="."):
    root = _set_root(root)
    try:
        cmd = f"cd { root } && git check-ignore --no-index { filename }"
        if len(_run_cmd(cmd)):
            return True
    except subprocess.CalledProcessError:
        pass
    return False

In [24]:
#hide
assert in_gitignore("project.sch-bak", root) == True
assert in_gitignore("project.sch", root) == False

In [25]:
#export
def run_docker_cmd(cmd,
                   workdir,
                   container,
                   v=False):
    """
    Run a command in a docker container under a UID mapped to the current user.
    This ensures that the current user is owner of any files created in the
    workdir.
    """
    UID = subprocess.check_output("id -u", shell=True).decode("utf-8").strip()
    docker_cmd = (f"docker run --rm -v { workdir }:/workdir --workdir=\"/workdir\" "
        f"{ container } "
        f"/bin/bash -c \"useradd --shell /bin/bash -u { UID } -o -c '' -m docker && "
        f"runuser docker -c '{ cmd }'\""
    )
    if v:
        print(docker_cmd)
    return subprocess.check_output(docker_cmd, stderr=subprocess.STDOUT, shell=True)

In [26]:
#export
def run_kibot_docker(config:Param(f"KiBot configuation file", str),
                     root:Param("project root directory", str)=".",
                     v:Param("verbose", bool)=False,
                     output:Param("output path relative to ROOT")="."):
    """
    Run KiBot in a local docker container.
    """
    root = _set_root(root)
    if os.path.abspath(output) == output:
        raise RuntimeError(f"OUTPUT cannot be an absolute path; it must be relative to ROOT={ root }.")

    cmd = (f"kibot -c { config } "
       f"-e { get_schematic_path(root)[len(root) + 1:] } "
       f"-b { get_board_path(root)[len(root) + 1:] } "
       f"-d { output }"
    )
    run_docker_cmd(cmd,
                   workdir=os.path.abspath(root),
                   container="setsoft/kicad_auto_test:latest",
                   v=v
    )

In [27]:
#export
def get_board_metadata(root="."):
    """Get metadata from the `*.kicad_pcb` board file.
    """
    root = _set_root(root)
    with open(get_board_path(root), 'r') as f:
        board = f.read()
    matches = re.search("\(title_block.*title (?P<title>[^\)]*)\)"
        ".*date (?P<date>[^\)]*)\)"
        ".*rev (?P<rev>[^\)]*)\)"
        ".*company (?P<company>[^\)]*)\)",
        board, re.DOTALL
    )
    return matches.groupdict()

In [28]:
get_board_metadata(root)

{'title': '"40-channel HV switching board"',
 'date': '2017-06-01',
 'rev': '3.1.1',
 'company': '"Sci-Bots Inc."'}

In [29]:
#export
def update_board_metadata(update_dict, root="."):
    """Update metadata in the `*.kicad_pcb` board file.
    """
    root = _set_root(root)
    with open(get_board_path(root), 'r') as f:
        board = f.read()
    
    for key, value in update_dict.items():
        if " " in value and not value.startswith('\"') and not value.endswith('\"'):
            value = '\"' + value + '\"'
        board, n = re.subn(f"(?s)(?P<pre>\(title_block.*{ key } )[^\)]*\)",
            f"\g<pre>{ value })",
            board
        )
        assert n == 1
    with open(get_board_path(root), 'w') as f:
        f.write(board)

In [30]:
get_board_metadata(root)

{'title': '"40-channel HV switching board"',
 'date': '2017-06-01',
 'rev': '3.1.1',
 'company': '"Sci-Bots Inc."'}

In [31]:
original_metadata = get_board_metadata(root)

# Test setting new metadata
new_metadata = {"title": '"new title"',
    "date": '"new date"',
    "rev": '"new rev"',
    "company": '"new company"'
}
update_board_metadata(new_metadata, root)
assert get_board_metadata(root) == new_metadata

# Restore original metadata
update_board_metadata(original_metadata, root)
assert get_board_metadata(root) == original_metadata

In [32]:
#export
def get_schematic_metadata(root, filename=None):
    """Get metadata from a `*.sch` schematic file.
    """
    if filename is None:
        filename = get_schematic_path(root)
    elif os.path.abspath(filename) != filename:
        filename = os.path.join(root, filename)
    
    with open(filename, 'r') as f:
        schematic = f.read()

    matches = re.search("Title (?P<Title>[^\n]*)\n"
        ".*Date (?P<Date>[^\n]*)\n"
        ".*Rev (?P<Rev>[^\n]*)\n"
        ".*Comp (?P<Comp>[^\n]*)\n",
        schematic, re.DOTALL
    )

    return matches.groupdict()

In [33]:
get_schematic_metadata(root)

{'Title': '"40-channel HV switching board"',
 'Date': '"2017-06-01"',
 'Rev': '"3.1.1"',
 'Comp': '"Sci-Bots Inc."'}

In [34]:
get_schematic_metadata(root, filename="switches_0-19.sch")

{'Title': '"40-channel HV switching board"',
 'Date': '"2017-06-01"',
 'Rev': '"3.1.1"',
 'Comp': '"Sci-Bots Inc."'}

In [35]:
#export
def update_schematic_metadata(update_dict:dict,      # keys/values to update
                              root:str=".",          # project root directory
                              all_sheets:bool=True): # update subsheets
    """Update metadata in a `*.sch` schematic file.
    """
    root = _set_root(root)
    
    if all_sheets:
        files = glob.glob(os.path.join(root, "*.sch"))
    else:
        files = [get_schematic_path(root)]
    
    for file in files:
        with open(file, 'r') as f:
            schematic = f.read()

        for key, value in update_dict.items():
            if not value.startswith('\"') and not value.endswith('\"'):
                value = '\"' + value + '\"'
            schematic, n = re.subn(f"(?s)(?P<pre>{ key } )([^\n]*)\n",
                f"\g<pre>{ value }\n",
                schematic
            )
            assert n == 1
        with open(file, 'w') as f:
            f.write(schematic)

In [36]:
original_metadata = get_schematic_metadata(root)
original_metadata

{'Title': '"40-channel HV switching board"',
 'Date': '"2017-06-01"',
 'Rev': '"3.1.1"',
 'Comp': '"Sci-Bots Inc."'}

In [37]:
# Test setting new metadata
new_metadata = {"Title": '"new title"',
    "Date": '"new date"',
    "Rev": '"new rev"',
    "Comp": '"new company"'
}
update_schematic_metadata(new_metadata, root, all_sheets=True)
assert get_schematic_metadata(root) == new_metadata
assert get_schematic_metadata(root, "switches_0-19.sch") == new_metadata

# Restore original metadata
update_schematic_metadata(original_metadata, root, all_sheets=True)
assert get_schematic_metadata(root) == original_metadata
assert get_schematic_metadata(root, "switches_0-19.sch") == original_metadata

In [38]:
#hide
notebook2script()

Converted 00_actions.ipynb.
Converted 01_test.ipynb.
Converted 02_utilities.ipynb.
Converted index.ipynb.
